# Functional test with FTDI - ADF4351
- https://www.analog.com/media/en/technical-documentation/data-sheets/ADF4351.pdf

In [1]:
import os, sys

def gen_relative_path(target_path): 
    
    def del_same_parents(target_path, current_path):
        if len(target_path) * len(current_path) > 0:
            if target_path[0] == current_path[0]:
                target_path.pop(0)
                current_path.pop(0)
                del_same_parents(target_path, current_path)
            
    current_path = os.getcwd().replace('\\', '/').split('/')
    target_path = target_path.replace('\\', '/').split('/')
    del_same_parents(target_path, current_path)
    
    return ['..'] * len(current_path) + target_path


def append_source_relative_path(source_relative_path):
    sys.path.append(os.sep.join(source_relative_path))

In [2]:
# paths = [' ']
# paths = [gen_relative_path(p) for p in paths]
# print(paths)

In [3]:
paths = [['..', '..', '..', '..', '..', '..', '..', '已完成', 'Bridges', 'bitbucket', 'github', 'codes'], 
         ['..', '..', '..', '..', '..', '..', 'Utilities', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', 'codes']]

for path in paths:
    append_source_relative_path(path)

In [4]:
%pylab inline

from utilities.adapters import peripherals
from signal_generators.adf435x import ADF4351 
from utilities.shift_register import ShiftRegister

Populating the interactive namespace from numpy and matplotlib


## Debug mode?

In [5]:
cls = ADF4351 

cls.DEBUG_MODE_SHOW_BUS_DATA = False         # whether to show bus data. 
cls.DEBUG_MODE_PRINT_REGISTER = False        # whether to print registers. 

## Generators and Modulators

In [6]:
with_hardware_device = True

if with_hardware_device:
    _clk = peripherals.Pin.get_Ftdi_pin(pin_id = 4)
    _data = peripherals.Pin.get_Ftdi_pin(pin_id = 1)
    _ss = peripherals.Pin.get_Ftdi_pin(pin_id = 3)
    # _ss2 = peripherals.Pin.get_Ftdi_pin(pin_id = 0)

    _spi = ShiftRegister(stb_pin = _ss, clk_pin = _clk, data_pin = _data, polarity = 0)
else:
    _spi = _ss = None  # using None for testing without actual hardware device.

adf = cls(_spi, _ss)

[3932160, 134252497, 50352066, 6292659, 10256444, 5767173]
[5767168, 134610937, 50352066, 6292659, 9208356, 5767173]


In [ ]:
set_1 = [3932160, 134252497, 50352066, 6292659, 10256444, 5767173]
set_2 = [5767168, 134610937, 50352066, 6292659, 9208356, 5767173]

In [13]:
adf.map.load_values(enumerate([3932160, 134252497, 50352066, 6292659, 10256444, 5767173]))
adf.write_all_registers()
# adf.print()

In [32]:
adf.map.load_values(enumerate([5767168, 134610937, 50352066, 6292659, 9208356, 5767173]))
adf.map._registers[1].print()
# adf.write_all_registers()
# adf.print()


<< REGISTER_1 >>      :  ('0x805fff9', '0b1000000001011111111111111001')
[ Reserved_29 ]     :  0
[ Phase_Adjust ]    :  0
[ Prescaler_Value ] :  1
[ Phase_Value ]     :  11
[ MOD ]             :  4095
[ Index ]           :  1


15

In [ ]:
[3932160, 134252497, 50352066, 6292659, 10256444, 5767173]
[5767168, 134610937, 50352066, 6292659, 9208356, 5767173]

In [25]:
import pandas as pd
import json


self = adf

adf.map.load_values(enumerate([3932160, 134252497, 50352066, 6292659, 10256444, 5767173]))
reg_values_1 = json.loads(adf.map.dumps())
reg_values_1['registers'][0]

{'name': 'REGISTER_0',
 'code_name': 'REGISTER_0',
 'address': 0,
 'description': 'REGISTER_0',
 'default_value': 0,
 'elements': [{'name': 'Reserved_31',
   'idx_lowest_bit': 31,
   'n_bits': 1,
   'value': 0,
   'read_only': True,
   'code_name': 'Reserved_31',
   'description': 'Reserved'},
  {'name': 'INT',
   'idx_lowest_bit': 15,
   'n_bits': 16,
   'value': 120,
   'read_only': False,
   'code_name': 'INT',
   'description': 'The 16 INT bits (Bits[DB30:DB15]) set the INT value, which determines the integer part of the feedback division factor. The INT value is used in Equation 1 (see the INT, FRAC, MOD, and R Counter Relationship section). Integer values from 23 to 65,535 are allowed for the 4/5 prescaler; for the 8/9 prescaler, the minimum integer value is 75.'},
  {'name': 'FRAC',
   'idx_lowest_bit': 3,
   'n_bits': 12,
   'value': 0,
   'read_only': False,
   'code_name': 'FRAC',
   'description': 'The 12 FRAC bits (Bits[DB14:DB3]) set the numerator of the fraction that is i

In [29]:
df1 = pd.DataFrame(reg_values_1['registers'][0]['elements'])
df1

,name,idx_lowest_bit,n_bits,value,read_only,code_name,description
0,Reserved_31,31,1,0,True,Reserved_31,Reserved
1,INT,15,16,120,False,INT,The 16 INT bits (Bits[DB30:DB15]) set the INT ...
2,FRAC,3,12,0,False,FRAC,The 12 FRAC bits (Bits[DB14:DB3]) set the nume...
3,Index,0,3,0,True,Index,"When Bits[C3:C1] are set to 000, Register 0 is..."


In [22]:
df1 = pd.DataFrame(reg_values_1['registers'])
df1

,name,code_name,address,description,default_value,elements
0,REGISTER_0,REGISTER_0,0,REGISTER_0,0,"[{'name': 'Reserved_31', 'idx_lowest_bit': 31,..."
1,REGISTER_1,REGISTER_1,1,REGISTER_1,0,"[{'name': 'Reserved_29', 'idx_lowest_bit': 29,..."
2,REGISTER_2,REGISTER_2,2,REGISTER_2,0,"[{'name': 'Reserved_31', 'idx_lowest_bit': 31,..."
3,REGISTER_3,REGISTER_3,3,REGISTER_3,0,"[{'name': 'Reserved_24', 'idx_lowest_bit': 24,..."
4,REGISTER_4,REGISTER_4,4,REGISTER_4,8388608,"[{'name': 'Reserved_24', 'idx_lowest_bit': 24,..."
5,REGISTER_5,REGISTER_5,5,REGISTER_5,1572864,"[{'name': 'Reserved_24', 'idx_lowest_bit': 24,..."


In [ ]:
registers_names = pd.DataFrame([(address, name) for (address, name, _) in self.address_name_values],
                               columns = ['address', 'register_name'])
df_1 = pd.DataFrame(set_1, columns = ['address', 'value'])
df_2 = pd.DataFrame(set_2, columns = ['address', 'value'])

df = pd.merge(registers_names, df_1, how = 'outer', on = ['address'])
df = pd.merge(df, df_2, how = 'outer', on = ['address'], suffixes = ('_1', '_2'))

df.drop(df.index[pd.isna(df.value_1) & pd.isna(df.value_2)], inplace = True)
df['different'] = (df.value_1 != df.value_2).astype(int)
df.sort_values(by = ['address'], inplace = True)
df.index = range(len(df))

return df

## Current Configuration

In [ ]:
adf.init()

In [ ]:
# adf.band_select_clock_divider._set_mode('LOW')

In [ ]:
adf.set_frequency(1.50e9, channel_resolution = 100e3)

In [ ]:
adf.write_all_registers()

In [ ]:
adf.current_dividers

In [ ]:
adf.freq_pfd

In [ ]:
adf.rf_n_divider.MOD

In [ ]:
# 1.5GHz
adf.set_dividers(*(2, 100, 2, 12000.0, 2))

In [ ]:
adf.rf_divider._set_divider(2)

In [ ]:
adf.band_select_clock_divider._refresh_divider()

In [ ]:
adf.current_configuration[0]

In [ ]:
reg_values = [reg.value for reg in adf.map._registers]
[hex(r) for r in reg_values]

In [ ]:

_spi = ShiftRegister(stb_pin = _ss, clk_pin = _clk, data_pin = _data, polarity = 0)

In [7]:
AD_default_reg_values = [0x320000, 0x8008011, 0x4E42, 0x4B3, 0x8C803C, 0x580005]
reg_values = AD_default_reg_values

In [ ]:
_clk = peripherals.Pin.get_Ftdi_pin(pin_id = 4)
_data = peripherals.Pin.get_Ftdi_pin(pin_id = 1)
_ss = peripherals.Pin.get_Ftdi_pin(pin_id = 3)
# _ss2 = peripherals.Pin.get_Ftdi_pin(pin_id = 0)

_spi = ShiftRegister(stb_pin = _ss, clk_pin = _clk, data_pin = _data, polarity = 0)

_spi = SPI(spi, ss, ss_polarity = ss_polarity)

In [9]:
reg_values = [0x2ee00000, 0x80587d1, 0x3320fc2, 0xe004b3, 0x901224, 0x400005]

In [13]:
reg_values = [3932160, 134250513, 20034, 1203, 10256444, 5767173]

In [16]:
import time
from array import array

for r in reg_values[::-1]:
#     print(hex(reg_values[i]), array('B', reg_values[i].to_bytes(4, 'big')))
    
    adf._write_register_value(r)
    time.sleep(0.1)
    
# adf._write_register_value(reg_values[0])
# time.sleep(0.1)
    
#     adf._write_register_value(reg_values[0])
#     if i in (1, 2):
#         adf._write_register_value(reg_values[0])
#         time.sleep(0.1)

In [ ]:
adf.write_all_registers()

In [ ]:
adf._initialize_registers()

In [ ]:
adf.enable(True)

In [ ]:
adf.init()

In [ ]:
freq_desired = 1.5e9

In [ ]:
adf.charge_pump._set_current(5)

adf.r_counter._set_divider(1)

adf.set_frequency(freq_desired)

In [ ]:
adf.current_dividers

In [ ]:
adf.current_configuration[0]

In [ ]:
freq_ref = 25e6
d_ref_doubler = 2
d_r_count = 1
d_ref_divider = 2
d_rf_divider = 2

d_n_divider  = freq_desired * d_rf_divider * d_ref_divider * d_r_count / (freq_ref * d_ref_doubler)

adf.set_dividers(*(d_ref_doubler, d_r_count, d_ref_divider, d_n_divider, d_rf_divider))
d_r_count , d_n_divider, adf.freq_pfd, adf.current_frequency

In [ ]:
adf.band_select_clock_divider._set_mode('LOW')

In [ ]:
adf.band_select_clock_divider._set_divider(1)

In [ ]:
adf.band_select_clock_divider._refresh_divider()

In [ ]:
adf.set_frequency(50.1e6)

In [ ]:
df_dividers, df_controls = adf.current_configuration
df_dividers

In [ ]:
df_controls

In [ ]:
adf.rf_n_divider.MOD

In [ ]:
adf.enable(True)

In [ ]:
# adf.current_dividers

In [ ]:
adf.init()

In [ ]:
adf.current_configuration[1]

In [ ]:
mode, required_divider, divider = adf.band_select_clock_divider._refresh_divider()
mode, required_divider, divider

In [ ]:
adf.rf_out.enable(True)

In [ ]:
adf.band_select_clock_divider._set_mode('LOW')

In [ ]:
adf.band_select_clock_divider.mode

In [ ]:
adf.band_select_clock_divider.divider

In [ ]:
adf.band_select_clock_divider.freq

In [ ]:
adf.band_select_clock_divider._set_divider(100)

In [ ]:
adf.band_select_clock_divider._refresh_divider()

In [ ]:
adf.current_configuration[0]

In [ ]:
adf.current_configuration[1]

In [ ]:
adf.init()

In [ ]:
# 1.5GHz
adf.set_dividers(*(2, 200, 2, 24000.0, 2))
adf.current_configuration[0]

In [ ]:
adf.freq_pfd, adf.band_select_clock_divider.mode, adf.band_select_clock_divider.divider, adf.freq_pfd / adf.band_select_clock_divider.divider

In [ ]:
# 1.5GHz
adf.set_dividers(*(2, 250, 2, 30004.0, 2))
adf.current_configuration[1]

In [ ]:
adf.freq_pfd, adf.band_select_clock_divider.mode, adf.band_select_clock_divider.divider, adf.freq_pfd / adf.band_select_clock_divider.divider

In [ ]:
# 1.5GHz
adf.set_dividers(*(2, 1, 2, 120.016, 2))
adf.current_frequency

In [ ]:
# 50MHz
adf.set_dividers(*(1, 1, 1, 128, 64))
adf.current_frequency

In [ ]:

adf.band_select_clock_divider._refresh_divider()

In [ ]:
# 50MHz
adf.set_dividers(*(2, 10, 2, 1280, 64))
adf.current_frequency

In [ ]:
adf.set_frequency(50e6)

In [ ]:
adf.current_dividers

In [ ]:
adf.band_select_clock_divider._set_mode('LOW')
adf.band_select_clock_divider._set_divider(21)

In [ ]:
# adf.current_dividers

In [ ]:
# adf.current_configuration[0]

In [ ]:
# adf.phase_frequency_detector._enable_cycle_slip_reduction(False)

In [ ]:
adf.band_select_clock_divider._set_mode('LOW')
adf.freq_pfd

In [ ]:
adf.band_select_clock_divider._set_mode('HIGH')
adf.freq_pfd

In [ ]:
adf.band_select_clock_divider._set_divider(26)

In [ ]:
adf.band_select_clock_divider._set_divider(200)

In [ ]:
adf.rf_out.set_output_power(5)

In [ ]:
adf.clock_divider._set_mode('FAST_LOCK_ENABLE')

In [ ]:
adf.clock_divider._set_divider(3)

In [ ]:
adf.band_select_clock_divider._source = adf.ref_divider

In [ ]:
adf.band_select_clock_divider._source = adf.r_counter

In [ ]:
mode, required_divider, divider = adf.band_select_clock_divider._refresh_divider()
mode, required_divider, divider

In [ ]:
adf.band_select_clock_divider.freq

In [ ]:
adf.rf_out.freq

In [ ]:
adf._confirm_double_buffer()

In [ ]:
# adf.init()
adf.r_counter._set_divider(1)
adf.set_dividers(*(2, 1, 2, 120.016, 2))

In [ ]:
adf.band_select_clock_divider._enable_band_select(True)

In [ ]:
# adf._write_element_by_name('Counter_Reset', 0)

In [ ]:
adf.rf_out.enable(False)

In [ ]:
adf.rf_out.enable(True)

In [ ]:
adf.enable_output(False)

In [ ]:
adf.enable_output(True)

In [ ]:
adf._enable_counters(True)
# adf._confirm_double_buffer()

In [ ]:
# adf.enable(True)

In [ ]:
RF_out_min = 2.2e9 / 64
RF_out_max = 4.4e9
RF_out_min, RF_out_max

In [ ]:
# adf.set_frequency(RF_out_min, channel_resolution = 100e3, rf_divider_as = None)
adf.set_frequency(35e6)
# adf.set_frequency(1.599899e9) 
# adf.set_frequency(RF_out_max, channel_resolution = 100e3, rf_divider_as = None)

# adf.phaser.set_phase(90)

df_dividers, df_controls = adf.current_configuration
df_dividers

In [ ]:
df_controls

In [ ]:
adf.freq_resolution

## Find Integer N-dividers for a Frequency

In [ ]:
adf.find_integer_N_dividers(freq_desired = 1.500e9,
                            ref_doubled_by_2 = True, ref_divided_by_2 = True, 
                            rf_divider_as = None, 
                            torance_hz = 1, freq_ref = 25e6)

In [ ]:
adf.find_integer_N_dividers(freq_desired = 50e6,
                            ref_doubled_by_2 = True, ref_divided_by_2 = True, 
                            rf_divider_as = None, 
                            torance_hz = 1, freq_ref = 25e6)

In [ ]:
adf.write_all_registers()

In [ ]:
adf.set_dividers(*(2, 500, 2, 64000.0, 64))
adf.rf_n_divider.freq
adf.current_configuration[0]

In [ ]:
adf.set_dividers(*(2, 500, 2, 60000.0, 2))
adf.current_configuration[0]

In [ ]:
adf.set_dividers(*(2, 125, 2, 15002.0, 2))
adf.current_configuration[0]

In [ ]:
adf.status

In [ ]:
adf.print()

In [ ]:
adf.init()

## Member functions test

In [ ]:
for f in dir(cls):
    if not f.startswith('_'):
        print('adf.{}()'.format(f))